# Azure ML - Compute Clusters

## Loguearse a Azure ML

Como las acciones que vamos a hacer por CLI o a través del SDK de Python necesitan una autentificación, primero vamos a loguearnos en Azure ML

### Login en Azure ML con el CLI de Azure ML

Para logearnos en Azure hacemos

In [ ]:
!az login

Se nos abrirá el navegador para logearnos

### Crear un cliente de Azure ML con el SDK de Python

Primero creamos dos variables con la ID de la suscripción y el grupo de recursos, como estos son datos personales, no los voy a poner aquí. Lo que voy a hacer es incluirlos en un archivo `.env` que no voy a subir a GitHub

```bash
AZURE_SUSCRIPION_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
AZURE_ML_RESOURCE_GRPU_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
```

Ahora para leerlos primero necesitasos tener instalado `dotenv` que lo hacemos mediante `pip install python-dotenv`

In [1]:
import os
import dotenv

dotenv.load_dotenv()

AZURE_SUSCRIPION_ID = os.getenv("AZURE_SUSCRIPION_ID")
AZURE_ML_RESOURCE_GRPU_ID = os.getenv("AZURE_ML_RESOURCE_GRPU_ID")


Ahora que tenemos estas variables creamos un cliente

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

workspace_name = "azure-ml-workspace-Python-SDK"

ml_client = MLClient(DefaultAzureCredential(), AZURE_SUSCRIPION_ID, AZURE_ML_RESOURCE_GRPU_ID, workspace_name)

## Compute Clusters

Vamos a volver a ver la arquitectura de `Azure ML`

![Azure ML architecture concepts](https://images.maximofn.com/azureml-architecture-concepts.webp)

Al igual que antes henmos creado un `Compute Instance`, ahora vamos a ver cómo crear un `Compute Cluster`

#### Crear un `Compute Cluster` desde la interfaz gráfica

En `Azure ML studio` deberíamos tener el `Workspace` `Azure ML Workspace GUI`, así que entramos en él y pinchamos en el botón `New` y seleccionamos `Compute cluster`
 * En localización seleccionamos la misma que antes que el `Workspace`, en mi caso `France Central`
 * En `Virtual Machine tier` podemos seleccionar `Dedicated` para que garantice que siempre esté disponible, aunque es más caro, o `Low priority` que es más barato pero puede ser que no esté disponible
 * Seleccionamos el tipo de máquina, si `CPU` o `GPU`, en mi caso selecciono `GPU` para poder entrenar modelos
 * Seleccionamos el tipo de `GPU` que queremos, espero que cuando leas esto los filtros sean mejores y puedas filtrar por `VRAM`, pero cuando lo estoy escribiendo no está ese filtro básico. En mi caso, como no voy a hacer entrenamientos muy grandes elijo la `Standard_NC4as_T4_v3` que tiene una `Tesla T4` con `16GB` de `VRAM`
 * Pulsamos `Next`
 * Le ponemos un nombre, en mi caso le pondré `compute-cluster-GUI`
 * Ponemos el número mínimo de nodos. En mi caso pongo 0, así cuando no esté en uso no me cobrarán
 * Ponemos el número máximo de nodos. En mi caso pongo 4, aunque para el uso que le voy a dar en este post con 1 me sobra
 * Seleccionamos los segundos para que se reduzca el número de nodos, en mi caso pongo 60 segundos
 * Pulsamos `Create` para crear el cluster, tardará unos minutos

Desafortunadamente por defecto no vamos a tener quota de máquinas con GPU. Por lo que aunque hayamos creado un cluster con GPU, no vamos a poder usarlo. Más adelante hablaremos de esto

Cuando se crea hay una zona llamada `Applications` donde podemos elegir si ejecutar el cluster en `JupyterLab`, `Jupyter`, `vscode` en la web o `vscode` en local.

Por otro lado, cuando se crea el cluster, se crea con el mínimo número de nodos que hayas configurado

#### Crear un `Compute cluster` con el CLI de Azure ML

Creamos la `Compute instance` con el siguiente comando

In [6]:
!GROUP='rg-azure-ml' && \
WORKSPACE='azure-ml-workspace-CLI' && \
NAME="compute-instance-CLI" && \
SIZE="Standard_E4ds_v4" && \
TYPE="AmlCompute" && \
IDLE_BEFORE_SCALE_DOWN="60" && \
MIN_NODES="0" && \
MAX_NODES="4" && \
az ml compute create --name $NAME --size $SIZE --type $TYPE --workspace-name $WORKSPACE --resource-group $GROUP --idle-time-before-scale-down $IDLE_BEFORE_SCALE_DOWN --min-instances $MIN_NODES --max-instances $MAX_NODES

{| Finished ..
  "enable_node_public_ip": true,
  "id": "/subscriptions/40018f20-6173-46c6-9434-ce9df3089dce/resourceGroups/rg-azure-ml/providers/Microsoft.MachineLearningServices/workspaces/azure-ml-workspace-CLI/computes/compute-instance-CLI",
  "idle_time_before_scale_down": 60,
  "location": "francecentral",
  "max_instances": 4,
  "min_instances": 0,
  "name": "compute-instance-CLI",
  "network_settings": {},
  "provisioning_state": "Succeeded",
  "resourceGroup": "rg-azure-ml",
  "size": "Standard_E4ds_v4",
  "ssh_public_access_enabled": true,
  "tier": "dedicated",
  "type": "amlcompute"
}


Desafortunadamente por defecto no vamos a tener quota de máquinas con GPU. Por lo que aunque hayamos creado un cluster con GPU, no vamos a poder usarlo. Más adelante hablaremos de esto

#### Crear un `Compute cluster` con el SDK de Python

Creamos el `Compute cluster` con el siguiente código. Lo llamo `compute-cluster-Python` y no `compute-cluster-Python-SDK` porque el nombre tienen que tener 24 caracteres como máximo. Además, este `Compute cluster` lo voy a crear con una GPU `Standard_NC8as_T4_v3` porque no me deja crear más `Compute instance`s con la GPU que he elegido antes en la misma suscripción

In [3]:
from azure.ai.ml.entities import AmlCompute

compute_cluster_name = "compute-cluster-Python"
size = "Standard_E4ds_v4"
idle_time_before_scale_down = "30"
type_compute = "amlcompute"
location = "francecentral"
min_nodes = 0
max_nodes = 4
tier="Dedicated"

compute_cluster = AmlCompute(
    name=compute_cluster_name, 
    type=type_compute,
    location=location,
    size=size, 
    min_instances=min_nodes,
    max_instances=max_nodes,
    idle_time_before_scale_down=idle_time_before_scale_down,
    tier=tier,
)

result = ml_client.begin_create_or_update(compute_cluster).result()

In [4]:
result

AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'compute-cluster-Python', 'description': None, 'tags': None, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/40018f20-6173-46c6-9434-ce9df3089dce/resourceGroups/rg-azure-ml/providers/Microsoft.MachineLearningServices/workspaces/azure-ml-workspace-Python-SDK/computes/compute-cluster-Python', 'Resource__source_path': '', 'base_path': '/home/maximofernandez/Documents/web/portafolio/posts', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x740169725750>, 'resource_id': None, 'location': 'francecentral', 'size': 'Standard_E4ds_v4', 'min_instances': 0, 'max_instances': 4, 'idle_time_before_scale_down': 30.0, 'identity': None, 'ssh_public_access_enabled': True, 'ssh_settings': None, 'network_settings': <azure.ai.ml.entities._compute.compute.NetworkSettings object at 0x74016a2c82d0>, 'tier': 'dedicated', 'enable_node_pub

Desafortunadamente por defecto no vamos a tener quota de máquinas con GPU. Por lo que aunque hayamos creado un cluster con GPU, no vamos a poder usarlo. Más adelante hablaremos de esto